In [1]:
#python 3.11
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import datetime
import ipynb
from dotenv import load_dotenv
import pyodbc

Recordar cambiar fechas limite en las queries

In [2]:
today = datetime.today().strftime('%Y-%m-%d')
current_month = datetime.today().strftime('%B')
current_month

'January'

In [3]:
#Query from Database
def db_connect(env_path, query_name, db_name, server_name='Server', user_name='User', user_password='Pass'):
    
    load_dotenv(env_path)

    server = os.getenv(server_name)
    database = os.getenv(db_name)
    user = os.getenv(user_name) 
    password = os.getenv(user_password)
    query = os.getenv(query_name)

    cnxn_string = (f"Driver={{SQL Server Native Client 11.0}};"
            f"Server={server};"
            f"Database={database};"
            f"UID={user};"
            f"PWD={password};")
    
    cnxn = pyodbc.connect(cnxn_string)
    output = pd.read_sql(query, cnxn)
    cnxn.close()

    return(output)


In [29]:
envi_path = 'cross.env'

In [30]:
MT = db_connect(envi_path, query_name='query_MT', db_name='Database_MT')


C:\Users\NicolasSilvaNash\AppData\Local\Temp\ipykernel_17640\246157772.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output = pd.read_sql(query, cnxn)


In [31]:
TC = db_connect(envi_path, query_name='query_TC', db_name='Database_TC')

C:\Users\NicolasSilvaNash\AppData\Local\Temp\ipykernel_17640\246157772.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output = pd.read_sql(query, cnxn)


In [32]:
MT.shape[0]

81165

In [33]:
TC.shape[0]

80587

In [34]:
MT.TransactionId = MT.TransactionId.astype('str')

In [35]:
a = TC.merge(MT, how='outer',  right_on= 'TransactionId', left_on = 'TrxReference')
a[a.TransactionId.isna()]

,FechaSinTiempo,Id_x,SenderId,ReceiverId,TrxAmount,TrxCurrencyCode,TrxType,TrxDate,TrxReference,SendingClerkOrBranchId,...,SenderPhoneNumber,senderCountry,receiverFirstName,receiverLastName,receiverDocument,receiverDocumentType,receiverPhoneNumber,receiverCountry,receiverBankAccountNumber,receiverBankAccountType
1,2023-11-01,512100.0,e3595301-99fd-49da-ee8b-08db3f0c2275,b757dc2a-4c31-4375-8486-08dbe6bd95ed,230.86,None,BANKACCOUNT,2023-11-01 17:00:04.918125,None,PAYSEND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
cross_df = MT.merge(TC, how='left', left_on = 'TransactionId', right_on = 'TrxReference')

In [37]:
df = cross_df[cross_df.TrxReference.isnull()].sort_values(by='NetAmountUSD', ascending=False).reset_index(drop=True)

In [38]:
df = cross_df[cross_df.TrxReference.isnull()].sort_values(by='NetAmountUSD', ascending=False).reset_index(drop=True)
df = df.drop(columns=['FechaSinTiempo', 'Id_y', 'SenderId',
       'ReceiverId', 'TrxAmount', 'TrxCurrencyCode', 'TrxType', 'TrxDate',
       'TrxReference', 'SendingClerkOrBranchId', 'SendingInstitutionId',
       'SendingIntitutionName', 'SendingInstitutionCountryCode',
       'ReceivingInstitutionId', 'ReceivingInstitutionName',
       'ReceivingInstitutionCountryCode', 'SenderName', 'SenderAddress',
       'SenderMethod', 'ReceiverName', 'ReceiverAddress', 'ReceiverMethod',
       'SenderAgentName', 'SenderAgentAddress', 'SenderAgentOccupation',
       'SenderAgentId', 'ReceiverAgentName', 'ReceiverAgentAddress',
       'ReceiverAgentId', 'ReceiverAgentOccupation', 'SenderSegment',
       'ReceiverSegment', 'PaymentStatus', 'ExchangeRate_y', 'TrxFeesAndTaxes',
       'TrxChannel', 'PromotionId', 'SenderAmount', 'SenderCurrencyCode',
       'ReceiverAmount', 'ReceiverCurrencyCode', 'DateCreated',
       'DateCompleted', 'TrxDescription', 'IsRejected', 'FileDateCreated',
       'FileNameSaved'])


In [39]:
df_TargetCountry = pd.DataFrame(df.TargetCountry.value_counts())
df_SourceCountry = pd.DataFrame(df.SourceCountry.value_counts())
df_Client = pd.DataFrame(df.Client.value_counts())
df_VendorCode = pd.DataFrame(df.VendorCode.value_counts())
df_TargetCurrency = pd.DataFrame(df.TargetCurrency.value_counts())

In [55]:
with pd.ExcelWriter(rf'Files\TXS que el collector no ve NOV23 al final del mes.xlsx') as writer:  
     df.to_excel(writer, sheet_name                    ='Transactions', index=False)
     df_TargetCountry.to_excel(writer, sheet_name      ='TargetCountry', index=False)
     df_SourceCountry.to_excel(writer, sheet_name      ='SourceCountry', index=False)
     df_Client.to_excel(writer, sheet_name             ='Client', index=False)
     df_VendorCode.to_excel(writer, sheet_name         ='VendorCode', index=False)
     df_TargetCurrency.to_excel(writer, sheet_name     ='TargetCurrency', index=False)


# Other cross

In [41]:
cross2_df = TC.merge(MT, how='left', left_on = 'TrxReference' , right_on = 'TransactionId')

In [42]:
cross2_df[cross2_df.TransactionId.isnull()]

,FechaSinTiempo,Id_x,SenderId,ReceiverId,TrxAmount,TrxCurrencyCode,TrxType,TrxDate,TrxReference,SendingClerkOrBranchId,...,SenderPhoneNumber,senderCountry,receiverFirstName,receiverLastName,receiverDocument,receiverDocumentType,receiverPhoneNumber,receiverCountry,receiverBankAccountNumber,receiverBankAccountType
1,2023-11-01,512100,e3595301-99fd-49da-ee8b-08db3f0c2275,b757dc2a-4c31-4375-8486-08dbe6bd95ed,230.86,None,BANKACCOUNT,2023-11-01 17:00:04.918125,None,PAYSEND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ANOMALY DETECTION

In [43]:
txs = MT[['CreatedAt', 'TransactionId', 'Client', 'VendorCode', 
       'CollectMethod', 'TargetCountry', 'TargetCurrency', 'NetAmountUSD',
       'SendingAmount', 'RecipientAmount', 'SourceCountry',  'SenderDocument',
       'senderCountry', 'receiverDocument', 'receiverCountry', 'receiverBankAccountNumber']]


## Feature engineering

In [44]:
def client_mean_amount_p_user(df, client_name, amount_column = 'NetAmountUSD') -> float:
   return( round(df[df.Client==client_name][amount_column].mean(), 2))
def client_median_amount_p_user(df, client_name, amount_column = 'NetAmountUSD') -> float:
   return( round(df[df.Client==client_name][amount_column].median(), 2))
def vendor_median_amount_p_user(df, vendor_name, amount_column = 'NetAmountUSD') -> float:
   return( round(df[df.VendorCode==vendor_name][amount_column].median(), 2))
def vendor_mean_amount_p_user(df, vendor_name, amount_column = 'NetAmountUSD') -> float:
   return( round(df[df.VendorCode==vendor_name][amount_column].mean(), 2))

In [ ]:
client_median_amounts = {}
for client in txs.Client.unique():
    median_amount = client_median_amount_p_user(txs, client)
    client_median_amounts[client] = median_amount
    
client_mean_amounts = {}
for client in txs.Client.unique():
    mean_amount = client_mean_amount_p_user(txs, client)
    client_mean_amounts[client] = mean_amount

vendor_median_amounts = {}
for vendor in txs.VendorCode.unique():
    median_amount = vendor_median_amount_p_user(txs, vendor)
    vendor_median_amounts[vendor] = median_amount
    
vendor_mean_amounts = {}
for vendor in txs.VendorCode.unique():
    mean_amount = vendor_mean_amount_p_user(txs, vendor)
    vendor_mean_amounts[vendor] = mean_amount


In [54]:
txs['meanClientAmount'] = txs.Client.apply(lambda x: client_mean_amounts[x])
txs['medianClientAmount'] = txs.Client.apply(lambda x: client_median_amounts[x])
txs['meanVendorAmount'] = txs.VendorCode.apply(lambda x: vendor_mean_amounts[x])
txs['medianVendorAmount'] = txs.VendorCode.apply(lambda x: vendor_median_amounts[x])
txs

C:\Users\NicolasSilvaNash\AppData\Local\Temp\ipykernel_17640\1763274754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txs['meanClientAmount'] = txs.Client.apply(lambda x: client_mean_amounts[x])
C:\Users\NicolasSilvaNash\AppData\Local\Temp\ipykernel_17640\1763274754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txs['medianClientAmount'] = txs.Client.apply(lambda x: client_median_amounts[x])
C:\Users\NicolasSilvaNash\AppData\Local\Temp\ipykernel_17640\1763274754.py:3: SettingWithCopyWarning: 
A v

,CreatedAt,TransactionId,Client,VendorCode,CollectMethod,TargetCountry,TargetCurrency,NetAmountUSD,SendingAmount,RecipientAmount,SourceCountry,SenderDocument,senderCountry,receiverDocument,receiverCountry,receiverBankAccountNumber,meanClientAmount,medianClientAmount,meanVendorAmount,medianVendorAmount
0,2023-11-01 00:01:26.200680,1203211,TENPO,RIPPLENET,BANK_ACCOUNT,PER,PEN,5.005415,5.005415,18.55,CHL,45284425,CHL,72854648,PER,01800000408252043601,95.80,42.21,291.09,124.76
1,2023-11-01 00:00:51.471453,1203209,TENPO,ITALCAMBIO,BANK_ACCOUNT,VEN,VEF,230.797074,234.367074,7890.47,CHL,280289124,CHL,V11043656,VEN,01020103960000718868,95.80,42.21,83.88,33.90
2,2023-11-01 00:07:32.438285,1203223,TENPO,COBRU,BANK_ACCOUNT,COL,COP,331.240106,334.810106,1338355.44,CHL,262033236,CHL,25180350,COL,72529348844,95.80,42.21,215.27,76.34
3,2023-11-01 00:01:15.483017,1203210,TENPO,ITALCAMBIO,BANK_ACCOUNT,VEN,VEF,29.911022,33.481022,1022.60,CHL,168238312,CHL,V26230521,VEN,01910115151000015553,95.80,42.21,83.88,33.90
4,2023-11-01 00:00:25.861100,1203200,SANTANDERCHL,RIPPLENET,BANK_ACCOUNT,PER,PEN,433.982407,433.982407,1600.00,CHL,00235123339,CHL,16424371,PER,30592492425078,833.52,434.68,291.09,124.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81160,2023-11-30 23:58:45.187408,1303758,PMI,EASYPAGOS,BANK_ACCOUNT,ECU,USD,24.700000,26.450000,24.70,ECU,2300175904,ECU,2300175904,ECU,0019001525,189.47,57.95,240.37,79.20
81161,2023-11-30 23:59:43.082470,1303754,TENPO,ITALCAMBIO,BANK_ACCOUNT,VEN,VEF,25.142473,28.712473,868.53,CHL,262390284,CHL,V19179407,VEN,01340001610012268513,95.80,42.21,83.88,33.90
81162,2023-11-30 23:58:56.827904,1303865,REMITLY,RADAR,BANK_ACCOUNT,CHL,CLP,196.870031,197.870031,170558.00,USA,1a75945189df6782104125e34c62aa2c,USA,55632448,CHL,2200322602,242.24,100.21,314.85,139.57
81163,2023-11-30 23:59:26.762266,1303783,REMITLY,BANCOINDUSTRIAL,BANK_ACCOUNT,ARG,ARS,30.791211,31.791211,25000.00,GBR,41ae31c5deac699858eb275628cc36d0,GBR,27359731693,ARG,0720206588000045151990,242.24,100.21,88.66,48.59


In [ ]:
anomalias = df[df.is_anomaly==-1].sort_values(by='Anomaly_scores', ascending=True)

## ML

In [ ]:
df = df.set_index('CreatedAt')
X = pd.get_dummies(df[['Client', 'VendorCode', 
       'CollectMethod', 'TargetCountry',  'NetAmountUSD',
       'SourceCountry']])

In [ ]:
from sklearn.ensemble import IsolationForest
IF = IsolationForest(random_state=0, contamination=0.1).fit(X)
df['Anomaly_scores']  = IF.decision_function(X)
df['is_anomaly']  = IF.predict(X)
df.sort_values(by='is_anomaly')

c:\Users\NicolasSilvaNash\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Id_x,TransactionId,Client,VendorCode,Status,CollectMethod,TargetCountry,TargetCurrency,NetAmountUSD,SendingAmount,...,receiverFirstName,receiverLastName,receiverDocument,receiverDocumentType,receiverPhoneNumber,receiverCountry,receiverBankAccountNumber,receiverBankAccountType,Anomaly_scores,is_anomaly
CreatedAt,,,,,,,,,,,,,,,,,,,,,
2023-11-22 17:31:27.990720,90BDB5C9-AD84-4C1A-85D5-5C106BA6FA2A,1277528,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,5441.022042,5441.022042,...,Mirko,Zlatar sepulveda,X3660543R,5,+34651310338,ESP,ES5000817440190002109412,1.0,-0.025595,-1
2023-11-30 23:58:41.534071,50660FCB-C876-46C2-881E-6EFD4D4AB257,1303765,PMI,EASYPAGOS,COMPLETED,BANK_ACCOUNT,ECU,USD,114.700000,116.450000,...,YAMILEX,RUGEL,3050276702,5,+5930981976611,ECU,18000379152,0.0,-0.027654,-1
2023-11-30 23:56:14.177609,F4F096DF-E3D0-4E79-AE9F-9DF59D851692,1303830,REMITLY,RADAR,COMPLETED,BANK_ACCOUNT,CHL,CLP,115.426671,116.426671,...,Guissella,Molfino Casanova,232360100,7,+56964214930,CHL,000212822250,0.0,-0.013053,-1
2023-11-30 23:58:44.382088,79E678B9-6714-4CD6-A640-CAD3549D0F53,1303760,PMI,EASYPAGOS,COMPLETED,BANK_ACCOUNT,ECU,USD,124.700000,126.450000,...,ANA,CABRERA,1150885463,5,+593985447590,ECU,2209758370,0.0,-0.027654,-1
2023-11-15 21:30:52.583383,F62DF185-1404-4143-AAE1-AFD019343236,1255845,REMITLY,RADAR,COMPLETED,BANK_ACCOUNT,CHL,CLP,125.202686,126.202686,...,Raul,Ordenes Gonzalez,131840985,7,+56977236578,CHL,777913184098,0.0,-0.005658,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30 23:57:51.909319,6854B272-B110-444B-87D1-B54974254857,1303770,REMITLY,BANCOINDUSTRIAL,COMPLETED,BANK_ACCOUNT,ARG,ARS,103.535373,104.535373,...,Franco,Rubino,20380692431,7,+34636507295,ARG,0720175888000025170740,NaN,0.055134,1
2023-11-08 13:50:25.290584,0AE12B9E-B7B4-4BB0-B322-4878B63A2B7B,1230662,REMITLY,BANCOINDUSTRIAL,COMPLETED,BANK_ACCOUNT,ARG,ARS,103.647108,104.647108,...,Clara Esther,Friedman Kaprivka,27164532955,7,+541145599082,ARG,0170143840000002757032,NaN,0.055134,1
2023-11-09 11:05:59.017965,11729D0E-A132-4924-A947-E05C853B9AA7,1234063,BANCO_ESTADO,ITALCAMBIO,COMPLETED,BANK_ACCOUNT,VEN,VEF,103.710000,103.710000,...,Ofelia Rosa,Bellorin Yanez,V11172318,5,+584126983802,VEN,01020635910100001534,NaN,0.070061,1


In [ ]:
from sklearn.neighbors import LocalOutlierFactor


In [ ]:

clf = LocalOutlierFactor(n_neighbors=10, contamination=0.1)
y_pred = clf.fit_predict(df[[ 'TransactionId', 'NetAmountUSD']])


In [ ]:
y_pred

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,
       -1,  1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1, -1,
        1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,
       -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [ ]:
X_scores = clf.negative_outlier_factor_

In [ ]:
X_scores

array([ -7.18707333,  -3.20993243,  -2.21628031,  -2.23659208,
        -2.78160919,  -1.87026054, -10.16121884,  -2.20894742,
        -2.30652326,  -1.72493372,  -1.80351878,  -1.51911786,
        -1.04619392,  -1.33224201,  -1.44947621,  -1.22879275,
        -1.01870741,  -1.35914448,  -0.98906458,  -0.99497464,
        -1.00888038,  -1.19481202,  -1.14497573,  -1.222631  ,
        -1.11798981,  -1.29558229,  -0.97504738,  -0.96113101,
        -0.98818487,  -0.94988807,  -0.97965568,  -1.2433701 ,
        -1.09522198,  -1.08916547,  -1.18382167,  -1.00670103,
        -1.03363399,  -0.95437039,  -1.08751145,  -1.11439433,
        -1.09925721,  -1.14932304,  -1.05087526,  -1.01302806,
        -1.02770136,  -1.01742549,  -0.92813006,  -1.00993445,
        -1.00554705,  -1.03226369,  -0.98156386,  -1.19919059,
        -1.15408574,  -1.01978838,  -1.0303056 ,  -1.10654376,
        -1.01601913,  -0.96741054,  -0.95711675,  -1.02986885,
        -1.01095988,  -0.96996458,  -4.33413683,  -1.02

In [ ]:
df = og_data.copy()
df = df.set_index('CreatedAt')

X = pd.get_dummies(df[['Client', 'VendorCode', 
       'CollectMethod', 'TargetCountry',  'NetAmountUSD',
       'SourceCountry']])


from sklearn.ensemble import IsolationForest
IF = IsolationForest(random_state=0, contamination=0.1).fit(X)

df['Anomaly_scores']  = IF.decision_function(X)
df['is_anomaly']  = IF.predict(X)
df.sort_values(by='is_anomaly')
anomalias = df[df.is_anomaly==-1].sort_values(by='Anomaly_scores', ascending=True)

df.to_excel(rf'IF anomalies - NOV 23.xlsx', index=False)